In [1]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import pandas as pd

## Разделим данные на разные атласы

In [2]:
data = np.load('./ihb.npy')

In [3]:
print("Shape full data:", data.shape)

Shape full data: (320, 10, 246)


In [4]:
# Функция для определения, содержат ли данные np.nan
def is_schaefer(sample):
    # Возвращает True, если в данных есть хотя бы один np.nan
    return np.isnan(sample).any()
    
# Получаем индексы для каждого раздела
brainnetome_indices = np.array([i for i, sample in enumerate(data) if not is_schaefer(sample)])
schaefer_indices = np.array([i for i, sample in enumerate(data) if is_schaefer(sample)])

# Разделяем данные
brainnetome_data = data[brainnetome_indices]
schaefer_data = data[schaefer_indices]

# Проверяем размеры разделенных наборов данных
print("Shape of Brainnetome data:", brainnetome_data.shape)
print("Shape of Schaefer200 data:", schaefer_data.shape)

Shape of Brainnetome data: (160, 10, 246)
Shape of Schaefer200 data: (160, 10, 246)


- 10 объектов, для каждого 16 снимков

- для каждого снимка 10 фоток (10 временных измерений)

- в каждом измерении 246 фичей

# I способ

## Кластеризация объектов для Brainnetome

Сделаем кластеризацию для 10и объектов

Обучим модели на каждой фиче, которые есть в выборке. 

Запишем все лейблы, а потом выберем самое частовстречающееся значение этих лейблов

In [5]:
all_labeling_brainnetome = []

for i in range(246): # i - итерация по каждой фиче из 246
    common_brain_region_data = brainnetome_data[:, :, i]
    model = make_pipeline(StandardScaler(), KMeans(n_clusters=10))
    model.fit(common_brain_region_data)

    cluster_distances = model.transform(common_brain_region_data)
    labeling_brainnetome = np.zeros(len(brainnetome_data), dtype=int)
    leftover_indexes = np.arange(len(brainnetome_data))
    for i in range(10):
        distances_from_current_cluster_center = cluster_distances[:, i]
        if len(distances_from_current_cluster_center) > 16:
            top16 = np.argpartition(distances_from_current_cluster_center, 16)[:16]
            labeling_brainnetome[leftover_indexes[top16]] = i
            cluster_distances = np.delete(cluster_distances, top16, axis=0)
            leftover_indexes = np.delete(leftover_indexes, top16)
        else:
            labeling_brainnetome[leftover_indexes] = i

    all_labeling_brainnetome.append(labeling_brainnetome)

In [6]:
# Инициализация нового массива для хранения результата
result_array = [[0] * 246 for _ in range(160)]

# Заполнение result_array
for i in range(246):
    for j in range(160):
        result_array[j][i] = all_labeling_brainnetome[i][j]

# Теперь result_array содержит 160 объектов, каждый из которых содержит 246 чисел

In [7]:
new_result_array_labeling_brainnetome = []

for label in result_array:
    counts = [label.count(i) for i in range(10)]  # Считаем количество для каждого числа от 0 до 9
    max_count = max(counts)  # Находим максимальное количество
    max_index = counts.index(max_count)  # Находим индекс максимального количества
    new_result_array_labeling_brainnetome.append(max_index)  # Добавляем индекс в новый массив
   

## Кластеризация объектов для Schaefer200 с заполненными нулями нанами

аналогично предыдущему пункту

In [8]:
all_labeling_schaefer = []

for i in range(246): 
    common_brain_region_data = schaefer_data[:, :, i]  # i - итерация по каждой фиче из 246
    common_brain_region_data = np.nan_to_num(common_brain_region_data) # заполним наны нулями
    model = make_pipeline(StandardScaler(), KMeans(n_clusters=10))
    model.fit(common_brain_region_data)

    cluster_distances = model.transform(common_brain_region_data)
    labeling_schaefer = np.zeros(len(schaefer_data), dtype=int)
    leftover_indexes = np.arange(len(schaefer_data))
    for i in range(10):
        distances_from_current_cluster_center = cluster_distances[:, i]
        if len(distances_from_current_cluster_center) > 16:
            top16 = np.argpartition(distances_from_current_cluster_center, 16)[:16]
            labeling_schaefer[leftover_indexes[top16]] = i
            cluster_distances = np.delete(cluster_distances, top16, axis=0)
            leftover_indexes = np.delete(leftover_indexes, top16)
        else:
            labeling_schaefer[leftover_indexes] = i

    all_labeling_schaefer.append(labeling_schaefer)

/home/vmnikitina/.local/lib/python3.10/site-packages/sklearn/base.py:1473: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/vmnikitina/.local/lib/python3.10/site-packages/sklearn/base.py:1473: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/vmnikitina/.local/lib/python3.10/site-packages/sklearn/base.py:1473: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/vmnikitina/.local/lib/python3.10/site-packages/sklearn/base.py:1473: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


In [9]:
# Инициализация нового массива для хранения результата
result_array = [[0] * 246 for _ in range(160)]

# Заполнение result_array
for i in range(246):
    for j in range(160):
        result_array[j][i] = all_labeling_schaefer[i][j]

# Теперь result_array содержит 160 объектов, каждый из которых содержит 246 чисел

new_result_array_labeling_schaefer = []

for label in result_array:
    counts = [label.count(i) for i in range(10)]  # Считаем количество для каждого числа от 0 до 9
    max_count = max(counts)  # Находим максимальное количество
    max_index = counts.index(max_count)  # Находим индекс максимального количества
    new_result_array_labeling_schaefer.append(max_index)  # Добавляем индекс в новый массив
 

## Кластеризация объектов для Schaefer200 с удаленными нанами

Вот тут стоит посмотреть, где и как нужно удалять и как потом итерировать

## Объединим результаты

In [10]:
# с заполненными нанми

# Инициализируем массив для хранения всех предсказаний
all_predictions = np.empty(data.shape[0], dtype=int)

# Заполняем массив предсказаниями
all_predictions[brainnetome_indices] = new_result_array_labeling_brainnetome
all_predictions[schaefer_indices] = new_result_array_labeling_schaefer

# Теперь all_predictions содержит предсказания для всех 320 сэмплов
print("All predictions:", all_predictions)

All predictions: [0 8 7 7 9 9 6 6 9 9 9 9 0 9 9 9 9 9 9 6 1 3 9 5 6 9 9 8 9 1 8 7 3 9 8 2 8
 6 8 8 8 4 9 8 8 9 1 8 9 4 0 8 0 7 8 7 8 7 7 7 7 7 5 2 6 7 7 6 1 6 6 5 1 6
 6 6 1 6 7 6 6 5 4 0 5 1 5 9 5 2 5 3 6 6 5 7 3 4 4 5 6 9 4 4 2 5 5 5 4 3 5
 3 4 9 9 4 4 2 5 4 2 1 9 4 4 6 8 0 4 1 9 4 3 3 0 3 3 6 3 1 3 3 9 3 3 3 2 2
 9 0 2 8 2 2 2 2 9 2 5 2 2 2 0 1 3 1 6 0 1 2 1 4 1 1 8 4 1 1 6 9 1 1 1 1 1
 0 1 1 4 9 0 3 1 0 0 8 9 0 0 0 7 0 0 0 6 6 9 0 0 1 4 0 0 0 3 8 9 1 4 1 0 1
 0 9 0 6 1 0 8 0 8 0 2 3 1 1 2 1 3 1 2 5 2 0 7 2 5 0 3 2 2 8 7 3 1 2 4 4 3
 3 4 6 5 3 4 4 5 9 5 7 5 9 4 1 5 5 5 6 6 5 5 6 0 2 3 6 8 7 6 6 2 1 9 7 8 7
 7 7 7 7 8 3 7 8 8 3 9 8 3 8 7 7 8 8 7 5 7 7 9 9]


In [11]:
pd.DataFrame({'prediction': all_predictions}).to_csv('submission_1.csv', index=False)

# II способ

## Кластеризация без разделения на атласы

In [12]:
all_labeling = []

for i in range(246): 
    common_brain_region_data = data[:, :, i]
    common_brain_region_data = np.nan_to_num(common_brain_region_data) # заполним наны нулями
    model = make_pipeline(StandardScaler(), KMeans(n_clusters=20))
    model.fit(common_brain_region_data)

    cluster_distances = model.transform(common_brain_region_data)
    labeling = np.zeros(len(data), dtype=int)
    leftover_indexes = np.arange(len(data))
    for i in range(20):
        distances_from_current_cluster_center = cluster_distances[:, i]
        if len(distances_from_current_cluster_center) > 16:
            top16 = np.argpartition(distances_from_current_cluster_center, 16)[:16]
            labeling[leftover_indexes[top16]] = i
            cluster_distances = np.delete(cluster_distances, top16, axis=0)
            leftover_indexes = np.delete(leftover_indexes, top16)
        else:
            labeling[leftover_indexes] = i

    all_labeling.append(labeling)

In [13]:
# Инициализация нового массива для хранения результата
result_array = [[0] * 246 for _ in range(320)]

# Заполнение result_array
for i in range(246):
    for j in range(320):
        result_array[j][i] = all_labeling[i][j]

# Теперь result_array содержит 320 объектов, каждый из которых содержит 246 чисел

new_result_array_labeling = []

for label in result_array:
    counts = [label.count(i) for i in range(10)]  # Считаем количество для каждого числа от 0 до 9
    max_count = max(counts)  # Находим максимальное количество
    max_index = counts.index(max_count)  # Находим индекс максимального количества
    new_result_array_labeling.append(max_index)  # Добавляем индекс в новый массив
 

In [14]:
pd.DataFrame({'prediction': new_result_array_labeling}).to_csv('submission_2.csv', index=False)

# ИДЕИ

- нужно попробовать другие варианты скалеров и параметры для них

- нужно попробовать метод с удаленными нанами для шафера